# Task 4

In [ ]:
import os
import pandas as pd

# Import the module
from UsefulFunctions import Optimizer_NonProsumer

### 1st step: determine the best SOC strategy

In [ ]:
def profits_from_SOC_strategy(SOC_strategy, negative=True):
    """
    Calculate profits based on the given SOC strategy using the actual price data.
    This function loads the price data, processes it by day, and calculates the profits
    for the battery operation using the Optimizer_NonProsumer function.
    The profit can be returned as a negative value to be minimized.

    Parameters:
    SOC_strategy (float): The state of charge strategy for the battery, [0.1, 2].
    negative (bool): If True, the profit will be returned as a negative value.

    Returns:
    float: The total profit calculated for the given SOC strategy.
    """
    # Load data
    file_P = os.path.join(os.getcwd(),'Elspotprices2nd.csv')
    df_prices = pd.read_csv(file_P)
    df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
    df_prices.rename(columns={'SpotPriceDKK': 'Price'}, inplace=True)

    df_prices = df_prices[
        (df_prices["HourUTC"].dt.year == 2024) & 
        (df_prices["HourUTC"].dt.month == 9)
    ]

    # Battery parameters
    battery_params = {
        'Pmax': 1,      # Power capacity in MW
        'Cmax': 2,     # Energy capacity in MWh
        'Cmin': 0.2,      # Minimum SOC (10%)
        'C_0': SOC_strategy,       # Initial SOC
        'C_n': SOC_strategy,       # Final SOC
        'n_c': 0.95,    # Charging efficiency
        'n_d': 0.95     # Discharging efficiency
    }

    # Initialize result
    profits = 0

    # Process data by days
    days = pd.to_datetime(df_prices['HourUTC'].dt.date.unique())
    for i, day in enumerate(days):
        # Start with 50% SOC for the first day, requierment from the assignment
        if i == 0:
            battery_params['C_0'] = 1
        
        # Filter data for the current day
        day_date = day.date()
        prices_day = df_prices[df_prices['HourUTC'].dt.date == day_date]
        
        # Make sure we have price data for this day
        if len(prices_day) == 0:
            continue
        
        # Extract data
        prices = prices_day['Price'].values
        
        # Optimize battery operation with price arbitrage
        # The goal is to charge when prices are low and discharge when prices are high
        profit_value, p_c_value, p_d_value, X_value = Optimizer_NonProsumer(battery_params, prices)
        
        # Calculate the battery net discharge.
        # Positive means discharging/selling, negative means charging/buying
        net_discharge = p_d_value - p_c_value
        
        # Calculate cost with battery
        day_profit = 0
        for i in range(len(net_discharge)):
            day_profit += net_discharge[i] * prices[i]
        
        profits += day_profit

    # If negative is True, return the negative profit for minimization
    if negative:
        return -profits
    else:
        return profits

# Find the optimal SOC strategy. 
# ie the maximum of the function over the range [0.2, 2].
from scipy.optimize import minimize_scalar

# Use the bounded method to find the maximum profit
# within the bounds [0.2, 2]
result = minimize_scalar(profits_from_SOC_strategy, bounds=(0.2, 2), method='bounded')

optimal_SOC_strategy = result.x
optimal_profit = -result.fun
print(f"Optimal SOC strategy: {optimal_SOC_strategy:.2f}")

In [ ]:
# Plotting the profit function
import matplotlib.pyplot as plt
import numpy as np

# Define the range of SOC strategies to evaluate
SOC_strategies = np.linspace(0.2, 2, 100)

# Calculate profits for the range of SOC strategies
profits = [profits_from_SOC_strategy(soc, negative=False) for soc in SOC_strategies]

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(SOC_strategies, profits, label="Profit vs SOC Strategy", color="blue", linewidth=2)

# Highlight the optimal SOC strategy
plt.axvline(optimal_SOC_strategy, color="red", linestyle="--", label=f"Optimal SOC: {optimal_SOC_strategy:.2f}")
plt.scatter(optimal_SOC_strategy, optimal_profit, color="red", label=f"Optimal Profit: {optimal_profit:.0f} DKK")

# Add labels, title, and legend
plt.title("Profit as a Function of SOC Strategy", fontsize=16)
plt.xlabel("SOC Strategy", fontsize=14)
plt.ylabel("Profit (DKK)", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)

# Show the plot
plt.show()

In [ ]:
# Load data
file_P = os.path.join(os.getcwd(),'Elspotprices2nd.csv')
df_prices = pd.read_csv(file_P)
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices.rename(columns={'SpotPriceDKK': 'Price'}, inplace=True)

df_prices = df_prices[
    (df_prices["HourUTC"].dt.year == 2024) & 
    (df_prices["HourUTC"].dt.month == 9)
]

# Final and initial SOC strategy
SOC_strategy = 0.2

# Battery parameters
battery_params = {
    'Pmax': 1,      # Power capacity in MW
    'Cmax': 2,     # Energy capacity in MWh
    'Cmin': 0.2,      # Minimum SOC (10%)
    'C_0': SOC_strategy,       # Initial SOC
    'C_n': SOC_strategy,       # Final SOC
    'n_c': 0.95,    # Charging efficiency
    'n_d': 0.95     # Discharging efficiency
}

# Initialize result dictionarie
profits = {'Actual': 0, 'Forcast': 0, 'Persistence': 0}

# Process data by days
days = pd.to_datetime(df_prices['HourUTC'].dt.date.unique())
for i, day in enumerate(days):
    # Start with 50% SOC for the first day
    if i == 0:
        battery_params['C_0'] = 1
    
    # Filter data for the current day
    day_date = day.date()
    prices_day = df_prices[df_prices['HourUTC'].dt.date == day_date]
    
    # Make sure we have price data for this day
    if len(prices_day) == 0:
        continue
    
    # Extract data
    prices = prices_day['Price'].values
    
    # Optimize battery operation with price arbitrage
    # The goal is to charge when prices are low and discharge when prices are high
    profit_value, p_c_value, p_d_value, X_value = Optimizer_NonProsumer(battery_params, prices)
    
    # Calculate the battery net discharge.
    # Positive means discharging/selling, negative means charging/buying
    net_discharge = p_d_value - p_c_value
    
    # Calculate cost with battery
    day_profit = 0
    for i in range(len(net_discharge)):
        day_profit += net_discharge[i] * prices[i]
    
    profits['Actual'] += day_profit

print("--- 4.1 and 4.2 results ---")
print("Profit with actual prices: ", int(profits['Actual']), "DKK")
print("Profit with forecast prices: ", int(profits['Forcast']), "DKK")
print("Profit with persistence prices: ", int(profits['Persistence']), "DKK")